In [3]:
! dir

 Volume in drive E is Storage
 Volume Serial Number is D098-4C8A

 Directory of E:\GitHub\data_science\notebooks

03/24/2018  04:32 PM    <DIR>          .
03/24/2018  04:32 PM    <DIR>          ..
03/24/2018  04:25 PM                 0 .gitkeep
03/24/2018  04:32 PM    <DIR>          .ipynb_checkpoints
03/24/2018  03:24 PM               555 random_forest.ipynb
               2 File(s)            555 bytes
               3 Dir(s)  1,858,646,945,792 bytes free
